### Required Libraries

In [1]:
# beautifulsoup4      4.9.3
# fake-useragent      0.1.11
# helium              3.0.5
# ipython             7.22.0
# matplotlib          3.3.4
# pandas              1.2.4
# Pillow              8.2.0
# requests            2.25.1
# tqdm                4.59.0

# !pip install beautifulsoup4
# !pip install fake-useragent
# !pip install helium
# !pip install ipython
# !pip install Pillow
# !pip install requests
# !pip install tqdm
# !pip install selenium
# !pip install webdriver-manager

# from webdriver_manager.chrome import ChromeDriverManager
# driver = webdriver.Chrome(ChromeDriverManager().install())

In [2]:
from helium import *
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm as tq
import time
import requests
import os
from fake_useragent import UserAgent

prod = "OUTDOOR_FURNITURE"

In [3]:
df_category = pd.read_csv(f'files/{prod}.csv')
df_category.head()

,product_name,product_link_name,link,major_category_name,major_category_link
0,OUTDOOR_FURNITURE,PATIO DINING FURNITURE,https://www.canadiantire.ca/en/outdoor-living/...,OUTDOOR FURNITURE,https://www.canadiantire.ca/en/outdoor-living/...
1,OUTDOOR_FURNITURE,PATIO LOUNGE FURNITURE,https://www.canadiantire.ca/en/outdoor-living/...,OUTDOOR FURNITURE,https://www.canadiantire.ca/en/outdoor-living/...
2,OUTDOOR_FURNITURE,GAZEBOS PERGOLAS AND CANOPIES,https://www.canadiantire.ca/en/outdoor-living/...,OUTDOOR FURNITURE,https://www.canadiantire.ca/en/outdoor-living/...
3,OUTDOOR_FURNITURE,PATIO TABLES,https://www.canadiantire.ca/en/outdoor-living/...,OUTDOOR FURNITURE,https://www.canadiantire.ca/en/outdoor-living/...
4,OUTDOOR_FURNITURE,PATIO SEATING,https://www.canadiantire.ca/en/outdoor-living/...,OUTDOOR FURNITURE,https://www.canadiantire.ca/en/outdoor-living/...


In [4]:
def update_link(link):
    split = link.split('?') 
    complement = 'https:'
    updated_link = complement + split[0]
    return updated_link

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

options = webdriver.ChromeOptions()
ua = UserAgent()

import random
randomlist = []
for i in range(0,5):
    n = random.randint(110, 150)
    randomlist.append(n/10)

df_list = []
for i in tq(range(len(df_category))):
    print(i)
    product_name = df_category['product_name'][i]
    class_name = df_category['product_link_name'][i]
    url = df_category['link'][i]
    
    print('Start fetching process for {}!'.format(class_name))
    
    if url[-3:] != '_no':
        number_of_pages = 5 
        url_list = [url + 'page={}'.format(i) for i in range(1, number_of_pages+1)]
    else:
        url_list = [url[:-3]]
    
    soup_list = []
    count = 0
    for page in tq(url_list):
        start = time.time()
        user_agent = ua.chrome
        options.add_argument('--user-agent="{}"'.format(user_agent))
        browser = start_chrome(page, headless=True, options=options)
        time.sleep(1)
        #webdriver.findElement(By.cssSelector("body")).sendKeys(Keys.ESCAPE);
        press(ESCAPE)
        time.sleep(5)
        soup = BeautifulSoup(browser.page_source, 'html.parser')

        #checker = soup.find_all('div', {'class':'srp-grid__item srp-grid__item_pcode'})
        checker = soup.find_all('div', {'class':'product-tile-srp__image-wrapper'})
        if len(checker) > 0:
            soup_list.append(soup)
            print('Page fetched: {}.'.format(page))
        elif (len(checker)==0) & (count < 2):
            count += 1
            print('Page not fetched: {}.'.format(page))
        else:
            print('Counter: ', count)
            print('Page not fetched: {}.'.format(page))
            print("No more pages avaiable")
            kill_browser()
            sleeping = random.choice(randomlist)
            time.sleep(sleeping)  
            end = time.time()
            time_elapsed = round(end - start, 1)
            print('Time elapsed: {}s'.format(time_elapsed))
            print('\n')
            try:
                kill_browser()
                print('Browser closed.')
            except:
                print('There were no browser open.')
                pass
            break

        try:
            kill_browser()
            print('Browser closed.')
        except:
            print('There were no browser open.')
            pass

        sleeping = random.choice(randomlist)
        time.sleep(sleeping)  
        end = time.time()
        time_elapsed = round(end - start, 1)
        print('Time elapsed: {}s'.format(time_elapsed))
    print('Fetching process for {} finished!'.format(class_name))
    print('Creating dataframe.')
    
    image_link_list = []
    original_name_list = []

    for soup in soup_list: 
        #products_list = soup.find_all('div', {'class':'srp-grid__item srp-grid__item_pcode'})
        products_list = soup.find_all('div', {'class':'product-tile-srp__image-wrapper'})
        for product in products_list:
            image = product.find('img')
            link = image['src']
            link = update_link(link)
            original_name = image['alt']
            image_link_list.append(link)
            original_name_list.append(original_name)

    df_temp = pd.DataFrame({'original_name': original_name_list, 'image_link': image_link_list})
    df_temp['class_name'] = class_name
    df_temp['product_name'] = product_name
    df_list.append(df_temp)
    print('Dataframe for {} created.\n\n\n\n'.format(class_name))

print('Creating final dataset.')
df_final = pd.concat(df_list)
df_final.drop_duplicates(subset='image_link', inplace=True)
df_final.reset_index(inplace=True)
del df_final['index']
print('Process finished. Printing final dataset.')
df_final

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

0
Start fetching process for PATIO DINING FURNITURE!
Page fetched: https://www.canadiantire.ca/en/outdoor-living/patio-furniture/dining.html?adlocation=LIT_PatioDiningFurniture_PatioFurnitureCLP_en;page=1.
Browser closed.



 20%|████████████████▊                                                                   | 1/5 [00:25<01:40, 25.13s/it]

Time elapsed: 25.1s
Page fetched: https://www.canadiantire.ca/en/outdoor-living/patio-furniture/dining.html?adlocation=LIT_PatioDiningFurniture_PatioFurnitureCLP_en;page=2.
Browser closed.



 40%|█████████████████████████████████▌                                                  | 2/5 [00:48<01:12, 24.14s/it]

...


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [01:37<00:24, 24.45s/it]

Time elapsed: 24.5s
Page fetched: https://www.canadiantire.ca/en/outdoor-living/patio-furniture/lounge-conversation-sets/canvas-breton-conversation-collection.html?adlocation=LIT_BretonConversationDiningCollection_PatioFurnitureCLP_en;page=5.
Browser closed.



100%|█████████████████████████████████████████████████████████████████████████████████| 23/23 [44:44<00:00, 116.72s/it]

Time elapsed: 23.3s
Fetching process for CANVAS BRETON CONVERSATION COLLECTION finished!
Creating dataframe.
Dataframe for CANVAS BRETON CONVERSATION COLLECTION created.




Creating final dataset.
Process finished. Printing final dataset.


,original_name,image_link,class_name,product_name
0,"CANVAS Minden Padded Sling Dining Set, 5-pc",https://canadiantire.scene7.com/is/image/Canad...,PATIO DINING FURNITURE,OUTDOOR_FURNITURE
1,"CANVAS Rockcliffe Casual Dining Set, 5-pc",https://canadiantire.scene7.com/is/image/Canad...,PATIO DINING FURNITURE,OUTDOOR_FURNITURE
2,"For Living Sling Folding Dining Set, 6-pc",https://canadiantire.scene7.com/is/image/Canad...,PATIO DINING FURNITURE,OUTDOOR_FURNITURE
3,For Living Bluebay Rectangular Patio Table,https://canadiantire.scene7.com/is/image/Canad...,PATIO DINING FURNITURE,OUTDOOR_FURNITURE
4,For Living Bluebay Rectangular Tile Patio Dini...,https://canadiantire.scene7.com/is/image/Canad...,PATIO DINING FURNITURE,OUTDOOR_FURNITURE
...,...,...,...,...
1202,"Hay Poly Tarp, 25 x 48-ft",https://canadiantire.scene7.com/is/image/Canad...,DECK BOXES AND OUTDOOR STORAGE,OUTDOOR_FURNITURE
1203,"Ice Rink Tarp, 20 x 40-ft",https://canadiantire.scene7.com/is/image/Canad...,DECK BOXES AND OUTDOOR STORAGE,OUTDOOR_FURNITURE
1204,"Certified Digital Camo Tarp, 5.9-ft x 8-ft",https://canadiantire.scene7.com/is/image/Canad...,DECK BOXES AND OUTDOOR STORAGE,OUTDOOR_FURNITURE
1205,"Certified Extreme Heavy Duty xPO Tarp, 12-ft x...",https://canadiantire.scene7.com/is/image/Canad...,DECK BOXES AND OUTDOOR STORAGE,OUTDOOR_FURNITURE


In [6]:
df_final.to_csv(f'files/{prod}_final.csv', index=False)
df_final.tail() 

,original_name,image_link,class_name,product_name
1202,"Hay Poly Tarp, 25 x 48-ft",https://canadiantire.scene7.com/is/image/Canad...,DECK BOXES AND OUTDOOR STORAGE,OUTDOOR_FURNITURE
1203,"Ice Rink Tarp, 20 x 40-ft",https://canadiantire.scene7.com/is/image/Canad...,DECK BOXES AND OUTDOOR STORAGE,OUTDOOR_FURNITURE
1204,"Certified Digital Camo Tarp, 5.9-ft x 8-ft",https://canadiantire.scene7.com/is/image/Canad...,DECK BOXES AND OUTDOOR STORAGE,OUTDOOR_FURNITURE
1205,"Certified Extreme Heavy Duty xPO Tarp, 12-ft x...",https://canadiantire.scene7.com/is/image/Canad...,DECK BOXES AND OUTDOOR STORAGE,OUTDOOR_FURNITURE
1206,"Certified Snow Camo Tarp, 5.9-ft x 8-ft",https://canadiantire.scene7.com/is/image/Canad...,DECK BOXES AND OUTDOOR STORAGE,OUTDOOR_FURNITURE


## Saving Images

In [7]:
file = f'files/{prod}_final.csv'
df_final = pd.read_csv(file)
df_final.tail()

,original_name,image_link,class_name,product_name
1202,"Hay Poly Tarp, 25 x 48-ft",https://canadiantire.scene7.com/is/image/Canad...,DECK BOXES AND OUTDOOR STORAGE,OUTDOOR_FURNITURE
1203,"Ice Rink Tarp, 20 x 40-ft",https://canadiantire.scene7.com/is/image/Canad...,DECK BOXES AND OUTDOOR STORAGE,OUTDOOR_FURNITURE
1204,"Certified Digital Camo Tarp, 5.9-ft x 8-ft",https://canadiantire.scene7.com/is/image/Canad...,DECK BOXES AND OUTDOOR STORAGE,OUTDOOR_FURNITURE
1205,"Certified Extreme Heavy Duty xPO Tarp, 12-ft x...",https://canadiantire.scene7.com/is/image/Canad...,DECK BOXES AND OUTDOOR STORAGE,OUTDOOR_FURNITURE
1206,"Certified Snow Camo Tarp, 5.9-ft x 8-ft",https://canadiantire.scene7.com/is/image/Canad...,DECK BOXES AND OUTDOOR STORAGE,OUTDOOR_FURNITURE


In [8]:
def get_full_link(link, image_number):    
    complement = '?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700'
    link = link[:-1]
    full_link = "{}{}{}".format(link, image_number, complement)
    return full_link

In [9]:
import os
total_images = 0
start = time.time()
links_not_fetched = []
for i in range(len(df_final)):
    product_name = df_final['class_name'][i]
    new_name = "{}_{}".format(product_name, i)
    original_name = df_final['original_name'][i]
    directory = f'images/{prod}'
    link = df_final['image_link'][i]
    
    try:
        os.makedirs('{}/{}'.format(directory, product_name))
    except OSError:
        pass
    
    print("\n\n Fetching images of: ", product_name, '\n')
    count = 0
    for i in range(1, 11):
        full_link = get_full_link(link, i)
        print('Link: ', full_link)
        file_name = '{}/{}/{}_photo_{}.jpg'.format(directory, product_name, new_name, i)
        img = requests.get(full_link)      
    
        if len(img.content) < 7000:
            print('No more product images.')
            print('Number of images saved: ', count)
            break
        else:
            count += 1
            total_images += 1
            print('Saving product image.')
            with open(file_name, 'wb') as f:
                f.write(img.content)
                time.sleep(0.5)
    if count == 0:
        links_not_fetched.append(full_link)
        print('\n\n', '########### \n', 'No image in the link: ', full_link, '\n #########','\n\n')
        
end = time.time()
time_elapsed = round(end - start, 1)
print('Time elapsed: {}s'.format(time_elapsed))
print('\n')
print('Total number of images: ', total_images)     



 Fetching images of:  PATIO DINING FURNITURE 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882274_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882274_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882274_3?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882274_4?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882274_5?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  4


 Fetching images of:  PATIO DINING FURNITURE 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882266_1?defaultImage

Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882276_7?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  6


 Fetching images of:  PATIO DINING FURNITURE 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882275_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882275_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882275_3?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  2


 Fetching images of:  PATIO DINING FURNITURE 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882057_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scen

Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0881787_7?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  6


 Fetching images of:  PATIO DINING FURNITURE 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882243_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882243_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882243_3?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882243_4?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882243_5?defaultImage=image_na_EN&fmt=jpg&fit=co

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882267_6?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882267_7?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882267_8?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882267_9?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882267_10?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  9


 Fetching images of:  PATIO DINING FURNITURE 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0881778_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882096_8?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882096_9?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  8


 Fetching images of:  PATIO DINING FURNITURE 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882231_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882231_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882231_3?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0882231_4?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=

...

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0600296_6?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0600296_7?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0600296_8?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  7


 Fetching images of:  DECK BOXES AND OUTDOOR STORAGE 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0600138_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0600138_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0600138_3?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=

Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0603081_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0603081_3?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0603081_4?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0603081_5?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0603081_6?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0603081_7?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.sc

Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0603526_5?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  4


 Fetching images of:  DECK BOXES AND OUTDOOR STORAGE 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0405001_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0405001_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0405001_3?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  2


 Fetching images of:  DECK BOXES AND OUTDOOR STORAGE 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0405084_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://c

Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0405110_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  1


 Fetching images of:  DECK BOXES AND OUTDOOR STORAGE 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0402506_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0402506_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0402506_3?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0402506_4?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  3


 Fetching images of:  DECK BOXES AND OUTDOOR STORAGE 

Link:  https://c

No more product images.
Number of images saved:  1


 Fetching images of:  DECK BOXES AND OUTDOOR STORAGE 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0405115_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0405115_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0405115_3?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of images saved:  2


 Fetching images of:  DECK BOXES AND OUTDOOR STORAGE 

Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0405014_1?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
Saving product image.
Link:  https://canadiantire.scene7.com/is/image/CanadianTire/0405014_2?defaultImage=image_na_EN&fmt=jpg&fit=constrain,1&wid=700&hei=700
No more product images.
Number of imag